In [1]:
#libraries
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
from PIL import Image
from pyometiff import OMETIFFReader
import tifffile as tif
import pandas as pd
import glob
from math import sqrt, exp, log
import io
import time
import subprocess
import re
from tqdm.notebook import tqdm
import fnmatch
import random
import json

In [2]:
exiftool_path = "exiftool"

In [10]:
##CREATE DIR
path0 = r"original"
path1 = r"raw"
path2 = r"processed"
path3 = r"thermal"

n = [path0, path1, path2, path3]

for i in list(n):
    if not os.path.exists(i):
        os.mkdir(i)

In [20]:
#RENAME FILES
i = 0
for filename in glob.glob(os.path.join(path0, "*.jpg")):
    if not re.search("img-\d\d\d" + re.escape(ext) + "$", filename):
        while True:
            newname = os.path.join(path0, "img-{:03d}{}".format(i, ".jpg"))
            if os.path.exists(newname):
                i+=1
            else:
                break
        #print('renaming "%s" to "%s"...' % (filename, newname))
        os.rename(filename, newname)

In [24]:
#ON FLIR CAMERAS PNG RAW ARRAYS REQUIRE BYTE INVERSION, TIFF DOES NOT
def raw_xtractor(filename):
    exiftool_path = "exiftool"
    raw_thermal_bytes = subprocess.check_output([exiftool_path, "-RawThermalImage", "-b", filename]) 
    thermal_img_stream = io.BytesIO(raw_thermal_bytes)
    thermal_img = Image.open(thermal_img_stream)
    thermal_np = np.array(thermal_img)
    if thermal_img.format == "PNG":
        thermal_np = np.floor(thermal_np/256) + np.mod(thermal_np, 256) * 256
        img = np.array(thermal_np, dtype = "uint16")
        img = Image.fromarray(img)
        basename = os.path.splitext(os.path.basename(filename))[0]
        img.save("raw/" + basename+".png")
    else:
        thermal_np = np.array(thermal_img)
        img = np.array(thermal_np, dtype = "uint16")
        img = Image.fromarray(img)
        basename = os.path.splitext(os.path.basename(filename))[0]
        img.save("raw/" + basename+".png")

In [25]:
#PROCESSING FOR SEGMENTATION
def imgprocessor(filename):
    for filename in glob.iglob(os.path.join(path1, "*.png")):
        img = cv2.imread(filename, -1)
        img_8bits = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, dtype = cv2.CV_8U)
        basename = os.path.splitext(os.path.basename(filename))[0]
        cv2.imwrite(os.path.join(path2, basename+"_"+"8bits"+".png"),img_8bits) 

In [26]:
original_images = []
for f in tqdm(glob.glob(os.path.join(path0, "*.jpg"))):
    or_img = cv2.imread(f, -1)
    raw_img = raw_xtractor(f)
    proc_img = imgprocessor(raw_img)
    original_images.append(np.array(or_img))

  0%|          | 0/2 [00:00<?, ?it/s]

In [114]:
#GET META
meta = {}
exiftool_path = "exiftool"
filename = "AF.jpg"

def extract_float(x):
  digits = re.findall(r"[-+]?\d*\.\d+|\d+", x)
  return float(digits[0])

process = subprocess.Popen([exiftool_path, filename], stdout=subprocess.PIPE, 
                           stderr=subprocess.STDOUT, universal_newlines=True) 
for tag in process.stdout:
     line = tag.strip().split(':')
     meta[line[0].strip()] = line[-1].strip()

for k,v in meta.items():
    print(k,':', v)

ExifTool Version Number : 12.30
File Name : AF.jpg
Directory : .
File Size : 288 KiB
File Modification Date/Time : 00
File Access Date/Time : 00
File Creation Date/Time : 00
File Permissions : -rw-rw-rw-
File Type : JPEG
File Type Extension : jpg
MIME Type : image/jpeg
JFIF Version : 1.01
Exif Byte Order : Little-endian (Intel, II)
Make : FLIR Systems AB
Camera Model Name : FLIR T530
Orientation : Horizontal (normal)
X Resolution : 72
Y Resolution : 72
Resolution Unit : inches
Software : 6.4.62
Modify Date : 39
Y Cb Cr Positioning : Centered
Exposure Time : 1/58
Exif Version : 0220
Create Date : 39
Components Configuration : -, Cr, Cb, Y
Subject Distance : 1 m
Focal Length : 10.3 mm
Image Temperature Max : 309
Image Temperature Min : 299
Flashpix Version : 0100
Color Space : sRGB
Exif Image Width : 640
Exif Image Height : 480
Digital Zoom Ratio : 1
Image Unique ID : A79450F20D14455B6F56717B6CCB7625
Compression : JPEG (old-style)
Thumbnail Offset : 1898
Thumbnail Length : 6398
Creator S

In [115]:
#TEMP CONVERSION
#meta FLIRONEPRO
def raw2temp(raw, 
             E=extract_float(meta["Emissivity"]), 
             OD=extract_float(meta["Object Distance"]), 
             RTemp=extract_float(meta["Reflected Apparent Temperature"]), 
             ATemp=extract_float(meta["Atmospheric Temperature"]), 
             IRWTemp=extract_float(meta["IR Window Temperature"]), 
             IRT=extract_float(meta["IR Window Transmission"]), 
             RH=extract_float(meta["Relative Humidity"]), 
             PR1=float(meta["Planck R1"]), 
             PB=float(meta["Planck B"]), 
             PF=float(meta["Planck F"]), 
             PO=float(meta["Planck O"]),
             PR2=float(meta["Planck R2"])):
    """
    convert raw values from the flir sensor to temperatures in C
    # this calculation has been ported to python from
    # https://github.com/gtatters/Thermimage/blob/master/R/raw2temp.R
    # a detailed explanation of what is going on here can be found there
    """

    # constants
    ATA1 = float(meta["Atmospheric Trans Alpha 1"])
    ATA2 = float(meta["Atmospheric Trans Alpha 2"])
    ATB1 = float(meta["Atmospheric Trans Beta 1"])
    ATB2 = float(meta["Atmospheric Trans Beta 2"])
    ATX =  float(meta["Atmospheric Trans X"])

    # transmission through window (calibrated)
    emiss_wind = 1 - IRT
    refl_wind = 0

    # transmission through the air
    h2o = (RH / 100) * exp(1.5587 + 0.06939 * (ATemp) - 0.00027816 * (ATemp) ** 2 + 0.00000068455 * (ATemp) ** 3)
    tau1 = ATX * exp(-sqrt(OD / 2) * (ATA1 + ATB1 * sqrt(h2o))) + (1 - ATX) * exp(
        -sqrt(OD / 2) * (ATA2 + ATB2 * sqrt(h2o)))
    tau2 = ATX * exp(-sqrt(OD / 2) * (ATA1 + ATB1 * sqrt(h2o))) + (1 - ATX) * exp(
        -sqrt(OD / 2) * (ATA2 + ATB2 * sqrt(h2o)))

    # radiance from the environment
    raw_refl1 = PR1 / (PR2 * (exp(PB / (RTemp + 273.15)) - PF)) - PO
    raw_refl1_attn = (1 - E) / E * raw_refl1
    raw_atm1 = PR1 / (PR2 * (exp(PB / (ATemp + 273.15)) - PF)) - PO
    raw_atm1_attn = (1 - tau1) / E / tau1 * raw_atm1
    raw_wind = PR1 / (PR2 * (exp(PB / (IRWTemp + 273.15)) - PF)) - PO
    raw_wind_attn = emiss_wind / E / tau1 / IRT * raw_wind
    raw_refl2 = PR1 / (PR2 * (exp(PB / (RTemp + 273.15)) - PF)) - PO
    raw_refl2_attn = refl_wind / E / tau1 / IRT * raw_refl2
    raw_atm2 = PR1 / (PR2 * (exp(PB / (ATemp + 273.15)) - PF)) - PO
    raw_atm2_attn = (1 - tau2) / E / tau1 / IRT / tau2 * raw_atm2
    raw_obj = (raw / E / tau1 / IRT / tau2 - raw_atm1_attn -
               raw_atm2_attn - raw_wind_attn - raw_refl1_attn - raw_refl2_attn)

    # temperature from radiance
    temp_celcius = PB / log(PR1 / (PR2 * (raw_obj + PO)) + PF) - 273.15
    return temp_celcius

In [116]:
raw2temp_vec = np.vectorize(raw2temp)

In [117]:
temp_images = []
for f in tqdm(glob.glob("*.png")):
    image = cv2.imread(f, -1)
 
    img_temp = raw2temp_vec(image)
    temp_images.append(np.array(img_temp)) 


  0%|          | 0/2 [00:00<?, ?it/s]

In [56]:
temp_images = 
for f in tqdm(glob.glob("*.jpg")):
    temp_images.append(np.array(extract_meta(f)))

  0%|          | 0/2 [00:00<?, ?it/s]

AttributeError: 'dict' object has no attribute 'append'